<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Extracting-KSS-from-Drive" data-toc-modified-id="Extracting-KSS-from-Drive-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Extracting KSS from Drive</a></span></li><li><span><a href="#Getting-HRV-Files" data-toc-modified-id="Getting-HRV-Files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting HRV Files</a></span><ul class="toc-item"><li><span><a href="#KSS-File-Names" data-toc-modified-id="KSS-File-Names-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>KSS File Names</a></span></li><li><span><a href="#Arranging-HRV-Dataframes" data-toc-modified-id="Arranging-HRV-Dataframes-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Arranging HRV Dataframes</a></span></li><li><span><a href="#Arranging-KSS-Dataframes" data-toc-modified-id="Arranging-KSS-Dataframes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Arranging KSS Dataframes</a></span></li></ul></li><li><span><a href="#Combining-Dataframes" data-toc-modified-id="Combining-Dataframes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Combining Dataframes</a></span></li></ul></div>

# Extracting KSS from Drive

In [10]:
from DDDS.drive import Drive
import pandas as pd
drive = Drive()

Connected successfully!


In [11]:
folders = drive.list('folder', entire_drive = True)
kss_folder = [folder['id'] for folder in folders if folder['name'] == 'KSS'][0]
kss_files = drive.list('csv', add_query=f"'{kss_folder}' in parents", entire_drive = True)
kss_files



[{'id': '1Zg6pUamNDmVD-e1DaZWJTirqqGrD3TPf',
  'name': 'kss_2021-11-24 15-36-04 e99.csv'},
 {'id': '1aEzG0t1vFUVBBSNB1oHmDv-WcDIJWwTq',
  'name': 'kss_2021-10-25 10-31-58 c27.csv'},
 {'id': '1YrK7iQWaYn8CcLQ3a_zSsPw3w2WPoyq3',
  'name': 'kss_2021-11-30 16-16-03 ece.csv'},
 {'id': '15BBZCsmorqdFqOuawOEu3Lw6S76pGhfn',
  'name': 'kss_2021-11-19 13-38-33 ece.csv'},
 {'id': '1L7PlZ7QsclQV9mwaA5R8Cw5RPmgVhnOr',
  'name': 'kss_2021-11-26 15-16-28 ea9.csv'},
 {'id': '1U5LhUajEjeWXt0wTDwojDkTXwFrQfGrb',
  'name': 'kss_2021-11-29 13-48-17 eca.csv'},
 {'id': '1CvTl_9POgJ6KaTwk7SFpDnxJ-Yjg-bdP',
  'name': 'kss_2021-11-02 15-23-39 1c3.csv'},
 {'id': '1kGTySEyRtVg89tnyeOTnf3L6oWSDJTP1',
  'name': 'kss_2021-11-04 11-01-00 eca.csv'},
 {'id': '1EzOp-y63AOzWZdvwYw8-R8TSjStq4KOJ',
  'name': 'kss_2021-11-22 11-26-24 ea9.csv'},
 {'id': '1QdQOWG9tB1NHcEJmeEG1tjo1B6MLFd-6',
  'name': 'kss_2021-11-26 12-34-32 1c3.csv'},
 {'id': '1x2WqV1UlXg4rN14_13c3kwOp8Mel-qGO',
  'name': 'kss_2021-11-17 10-58-38 e99.csv'},

In [13]:
kss_dfs_drive = {}

for file in kss_files:
    df = pd.read_csv(drive.download(file['id']))
    df.drop('Unnamed: 0', inplace=True, axis=1)

    kss_dfs_drive[file['name']] = df



Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Progress: |█████████████████████████████████████████

dict_keys(['kss_2021-11-24 15-36-04 e99.csv', 'kss_2021-10-25 10-31-58 c27.csv', 'kss_2021-11-30 16-16-03 ece.csv', 'kss_2021-11-19 13-38-33 ece.csv', 'kss_2021-11-26 15-16-28 ea9.csv', 'kss_2021-11-29 13-48-17 eca.csv', 'kss_2021-11-02 15-23-39 1c3.csv', 'kss_2021-11-04 11-01-00 eca.csv', 'kss_2021-11-22 11-26-24 ea9.csv', 'kss_2021-11-26 12-34-32 1c3.csv', 'kss_2021-11-17 10-58-38 e99.csv', 'kss_2021-10-26 13-32-50 c27.csv', 'kss_2021-12-01 13-20-00 e99.csv', 'kss_2021-11-24 11-01-03 eca.csv', 'kss_2021-11-18 13-21-41 e99.csv', 'kss_2021-12-07 14-35-11 ea9.csv', 'kss_2021-10-28 15-04-13 1c3.csv', 'kss_2021-11-05 13-59-29 ea9.csv'])

# Getting HRV Files

In [ ]:
hrv_list = [('28_10_2021', '844'),
 ('22_11_2021', 'eb0'),
 ('25_11_2021', 'ed2'),
 ('17_11_2021', 'e99'),
 ('25_10_2021', 'c27'),
 ('04_11_2021', 'ed2'),
 ('01_12_2021', 'e99'),
 ('24_11_2021', 'e99'),
 ('07_12_2021', 'ea9'),
 ('30_11_2021', 'ece'),
 ('30_11_2021', 'eb0'),
 ('05_11_2021', 'ed2'),
 ('22_11_2021', 'ea9'),
 ('24_11_2021', '038'),
 ('02_11_2021', '1c3'),
 ('29_11_2021', 'eca'),
 ('05_11_2021', 'ea9'),
 ('28_10_2021', '1c3'),
 ('26_10_2021', 'c04'),
 ('02_11_2021', 'eb0'),
 ('26_10_2021', 'c27'),
 ('19_11_2021', 'ece'),
 ('28_10_2021', '982'),
 ('24_11_2021', 'eca'),
 ('26_11_2021', '1c3'),
 ('25_11_2021', '038'),
 ('26_11_2021', 'ea9'),
 ('26_11_2021', 'ed2'),
 ('27_10_2021', '844'),
 ('04_11_2021', 'eca'),
 ('18_11_2021', 'e99'),
 ('25_10_2021', 'c04'),
 ('29_10_2021', '87e'),
 ('27_10_2021', '87e'),
 ('29_10_2021', '982')]

In [25]:
results_hrv = []
results_kss = []

for file in hrv_list:
    for key in kss_dfs_drive.keys():
        if '-'.join(file[0].split('_')[::-1]) in key and file[1] in key:
            results_hrv.append(file)
            results_kss.append(key)
            
results_hrv

[('17_11_2021', 'e99'),
 ('25_10_2021', 'c27'),
 ('01_12_2021', 'e99'),
 ('24_11_2021', 'e99'),
 ('07_12_2021', 'ea9'),
 ('30_11_2021', 'ece'),
 ('22_11_2021', 'ea9'),
 ('02_11_2021', '1c3'),
 ('29_11_2021', 'eca'),
 ('05_11_2021', 'ea9'),
 ('28_10_2021', '1c3'),
 ('26_10_2021', 'c27'),
 ('19_11_2021', 'ece'),
 ('24_11_2021', 'eca'),
 ('26_11_2021', '1c3'),
 ('26_11_2021', 'ea9'),
 ('04_11_2021', 'eca'),
 ('18_11_2021', 'e99')]

## KSS File Names

In [26]:
results_kss

['kss_2021-11-17 10-58-38 e99.csv',
 'kss_2021-10-25 10-31-58 c27.csv',
 'kss_2021-12-01 13-20-00 e99.csv',
 'kss_2021-11-24 15-36-04 e99.csv',
 'kss_2021-12-07 14-35-11 ea9.csv',
 'kss_2021-11-30 16-16-03 ece.csv',
 'kss_2021-11-22 11-26-24 ea9.csv',
 'kss_2021-11-02 15-23-39 1c3.csv',
 'kss_2021-11-29 13-48-17 eca.csv',
 'kss_2021-11-05 13-59-29 ea9.csv',
 'kss_2021-10-28 15-04-13 1c3.csv',
 'kss_2021-10-26 13-32-50 c27.csv',
 'kss_2021-11-19 13-38-33 ece.csv',
 'kss_2021-11-24 11-01-03 eca.csv',
 'kss_2021-11-26 12-34-32 1c3.csv',
 'kss_2021-11-26 15-16-28 ea9.csv',
 'kss_2021-11-04 11-01-00 eca.csv',
 'kss_2021-11-18 13-21-41 e99.csv']

## Arranging HRV Dataframes

In [27]:
from DDDS.hrv import HRV
hrv = HRV()

Connected successfully!


In [28]:
dataframes = hrv.get_dataframes()

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [29]:
def get_hrv_id(date, driver):
    """
    Returns key in hrv.dataframes dictionary corresponding to date and driver
    """
    for key in dataframes.keys():
        if date in key and driver in key:
            return key

In [32]:
key = get_hrv_id(*results_hrv[0])
key
dataframes[key]

,Timestamp_Google,Timestamp_Device,Device_id,Heart_Rate,RR_rate,Garmin
0,2021-11-17 09:58:22.566,2021-11-17 09:58:22.568,F1:4F:D8:DD:6A:C6,62,[619],1
1,2021-11-17 09:58:22.774,2021-11-17 09:58:22.771,CA:93:07:67:1E:FF,68,"[897, 905, 878, 904]",1
2,2021-11-17 09:58:22.807,2021-11-17 09:58:22.818,FD:9E:84:03:87:71,66,[],1
3,2021-11-17 09:58:23.293,2021-11-17 09:58:23.286,FD:9E:84:03:87:71,66,[],1
4,2021-11-17 09:58:23.498,2021-11-17 09:58:23.505,F1:4F:D8:DD:6A:C6,62,"[1016, 773]",1
...,...,...,...,...,...,...
12634,2021-11-17 11:05:03.944,2021-11-17 11:05:03.954,F1:4F:D8:DD:6A:C6,63,"[614, 293]",1
12635,2021-11-17 11:05:04.022,2021-11-17 11:05:04.032,FD:9E:84:03:87:71,61,[],1
12636,2021-11-17 11:05:04.542,2021-11-17 11:05:04.557,FD:9E:84:03:87:71,60,[],1
12637,2021-11-17 11:05:04.945,2021-11-17 11:05:04.963,F1:4F:D8:DD:6A:C6,63,[],1


In [36]:
# Get a list of HRV keys corresponding to Annotation DF
keys = []
for date, driver in results_hrv:
    keys.append(get_hrv_id(date, driver))
hrv_dfs_ordered = [dataframes[key] for key in keys]


,Timestamp_Google,Timestamp_Device,Device_id,Heart_Rate,RR_rate,Garmin
0,2021-11-17 09:58:22.566,2021-11-17 09:58:22.568,F1:4F:D8:DD:6A:C6,62,[619],1
1,2021-11-17 09:58:22.774,2021-11-17 09:58:22.771,CA:93:07:67:1E:FF,68,"[897, 905, 878, 904]",1
2,2021-11-17 09:58:22.807,2021-11-17 09:58:22.818,FD:9E:84:03:87:71,66,[],1
3,2021-11-17 09:58:23.293,2021-11-17 09:58:23.286,FD:9E:84:03:87:71,66,[],1
4,2021-11-17 09:58:23.498,2021-11-17 09:58:23.505,F1:4F:D8:DD:6A:C6,62,"[1016, 773]",1
...,...,...,...,...,...,...
12634,2021-11-17 11:05:03.944,2021-11-17 11:05:03.954,F1:4F:D8:DD:6A:C6,63,"[614, 293]",1
12635,2021-11-17 11:05:04.022,2021-11-17 11:05:04.032,FD:9E:84:03:87:71,61,[],1
12636,2021-11-17 11:05:04.542,2021-11-17 11:05:04.557,FD:9E:84:03:87:71,60,[],1
12637,2021-11-17 11:05:04.945,2021-11-17 11:05:04.963,F1:4F:D8:DD:6A:C6,63,[],1


## Arranging KSS Dataframes

In [41]:
kss_df_list = []
for key, val in kss_dfs_drive.items():
    kss_df_list.append(val)
    


,time_points,kss_value
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
2598,2598,2.0
2599,2599,2.0
2600,2600,2.0
2601,2601,2.0


# Combining Dataframes

In [60]:
import datetime as dt
import numpy as np
combined_dfs = []

# Combining KSS & HRV Dataframes...

for kss, hrv in zip(kss_df_list, hrv_dfs_ordered):
    
    # preprocessing hrv dataframes
    hrv['Timestamp_Google'] = hrv['Timestamp_Google'].dt.round('1s')
    hrv['time_points'] = hrv['Timestamp_Google'] - hrv['Timestamp_Google'].iloc[0]
    hrv['time_points']= hrv['time_points']/np.timedelta64(1,'s')
    hrv['time_points'] = hrv["time_points"].round().astype(int) 
    
    kss_start = kss[kss['kss_value'] != 0].time_points.iloc[0]
    hrv = hrv[hrv['time_points'] >=  kss_start]
    
    hrv['time_points'] = hrv['Timestamp_Google'] - hrv['Timestamp_Google'].iloc[0]
    hrv['time_points']= hrv['time_points']/np.timedelta64(1,'s')
    hrv['time_points'] = hrv["time_points"].round().astype(int) 
    
    df = hrv.merge(kss, on='time_points', how='left')
    
    df.drop('Timestamp_Device',inplace=True, axis=1, errors='ignore')
    df.drop('Device_id',inplace=True, axis=1, errors='ignore')
    
    df = df[df['kss_value'].notna()]

    combined_dfs.append(df)

/tmp/ipykernel_28253/246605520.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hrv['time_points'] = hrv['Timestamp_Google'] - hrv['Timestamp_Google'].iloc[0]
/tmp/ipykernel_28253/246605520.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hrv['time_points']= hrv['time_points']/np.timedelta64(1,'s')
/tmp/ipykernel_28253/246605520.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

/tmp/ipykernel_28253/246605520.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hrv['time_points'] = hrv["time_points"].round().astype(int)
/tmp/ipykernel_28253/246605520.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hrv['time_points'] = hrv['Timestamp_Google'] - hrv['Timestamp_Google'].iloc[0]
/tmp/ipykernel_28253/246605520.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

/tmp/ipykernel_28253/246605520.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hrv['time_points']= hrv['time_points']/np.timedelta64(1,'s')
/tmp/ipykernel_28253/246605520.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hrv['time_points'] = hrv["time_points"].round().astype(int)


In [63]:
combined_dfs[4].iloc[200:]

,Timestamp_Google,Heart_Rate,RR_rate,Garmin,time_points,kss_value
200,2021-12-07 13:37:07,104,[],1,49,3.0
201,2021-12-07 13:37:08,98,"[561, 529]",1,50,3.0
202,2021-12-07 13:37:08,106,"[598, 622]",1,50,3.0
203,2021-12-07 13:37:08,59,"[756, 503]",1,50,3.0
204,2021-12-07 13:37:08,106,[],1,50,3.0
...,...,...,...,...,...,...
15564,2021-12-07 14:40:48,95,[],1,3870,9.0
15565,2021-12-07 14:40:48,91,[687],1,3870,9.0
15566,2021-12-07 14:40:48,95,[],1,3870,9.0
15567,2021-12-07 14:40:48,100,"[592, 620]",1,3870,9.0
